In [32]:
import numpy as np
import pandas as pd
from datetime import timedelta
import glob
import os

import warnings
warnings.filterwarnings(action='ignore')

### 1. 서울시 5대 범죄 발생현황 연도별 통계(2015~2019)

In [2]:
file = pd.read_csv('raw/서울시 5대 범죄 발생현황 연도별 통계(2015~2019).txt', delimiter = '\t')
file

,기간,자치구,합계,합계.1,살인,살인.1,강도,강도.1,강간강제추행,강간강제추행.1,절도,절도.1,폭력,폭력.1
0,기간,자치구,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
1,2015,합계,"126,401","82,680",163,156,276,257,"5,449","5,069","55,307","21,842","65,206","55,356"
2,2015,종로구,"4,705","4,430",6,8,11,13,211,"1,139","2,184","1,145","2,293","2,125"
3,2015,중구,"4,954","2,942",3,2,9,6,170,111,"2,548",859,"2,224","1,964"
4,2015,용산구,"3,820","2,483",5,5,14,14,194,173,"1,557",587,"2,050","1,704"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2019,관악구,"5,328","3,810",13,10,10,11,408,335,"2,223","1,085","2,674","2,369"
127,2019,서초구,"5,542","3,750",5,5,7,5,616,412,"2,270","1,118","2,644","2,210"
128,2019,강남구,"7,304","5,069",5,3,5,6,666,562,"2,970","1,339","3,658","3,159"
129,2019,송파구,"5,698","3,799",7,8,10,10,273,230,"2,416","1,016","2,992","2,535"


In [3]:
# 컬럼명 변경
new_file = file.rename(columns={'합계' : '발생_합계',
                                '합계.1' : '검거_합계',
                                '살인' : '살인_발생',
                                '살인.1' : '살인_검거',
                                '강도' : '강도_발생',
                                '강도.1' : '강도_검거',
                                '강간강제추행' : '강간강제추행_발생',
                                '강간강제추행.1' : '강간강제추행_검거',
                                '절도' : '절도_발생',
                                '절도.1' : '절도_검거',
                                '폭력' : '폭력_발생',
                                '폭력.1' : '폭력_검거'})
new_file.head()

,기간,자치구,발생_합계,검거_합계,살인_발생,살인_검거,강도_발생,강도_검거,강간강제추행_발생,강간강제추행_검거,절도_발생,절도_검거,폭력_발생,폭력_검거
0,기간,자치구,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
1,2015,합계,"126,401","82,680",163,156,276,257,"5,449","5,069","55,307","21,842","65,206","55,356"
2,2015,종로구,"4,705","4,430",6,8,11,13,211,"1,139","2,184","1,145","2,293","2,125"
3,2015,중구,"4,954","2,942",3,2,9,6,170,111,"2,548",859,"2,224","1,964"
4,2015,용산구,"3,820","2,483",5,5,14,14,194,173,"1,557",587,"2,050","1,704"


In [4]:
# row 0 제거, 자치구==합계인 row 삭제
## 발생_합계, 검거_합계 컬럼 삭제해도 괜찮음 : 발생 컬럼의 총합이랑 같은 것임.
idx = new_file[new_file['자치구']=='자치구'].index | new_file[new_file['자치구']=='합계'].index
new_file = new_file.drop(idx)
new_file

,기간,자치구,발생_합계,검거_합계,살인_발생,살인_검거,강도_발생,강도_검거,강간강제추행_발생,강간강제추행_검거,절도_발생,절도_검거,폭력_발생,폭력_검거
2,2015,종로구,"4,705","4,430",6,8,11,13,211,"1,139","2,184","1,145","2,293","2,125"
3,2015,중구,"4,954","2,942",3,2,9,6,170,111,"2,548",859,"2,224","1,964"
4,2015,용산구,"3,820","2,483",5,5,14,14,194,173,"1,557",587,"2,050","1,704"
5,2015,성동구,"3,358","2,123",4,4,9,8,126,119,"1,607",597,"1,612","1,395"
6,2015,광진구,"5,909","3,707",4,4,14,26,240,220,"3,026","1,277","2,625","2,180"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2019,관악구,"5,328","3,810",13,10,10,11,408,335,"2,223","1,085","2,674","2,369"
127,2019,서초구,"5,542","3,750",5,5,7,5,616,412,"2,270","1,118","2,644","2,210"
128,2019,강남구,"7,304","5,069",5,3,5,6,666,562,"2,970","1,339","3,658","3,159"
129,2019,송파구,"5,698","3,799",7,8,10,10,273,230,"2,416","1,016","2,992","2,535"


In [11]:
# 기간 컬럼 타입 변경 : str -> int
new_file = new_file.astype({'기간' : 'int64'})
new_file

# 2017년 이후 데이터 분리
df_2016 = new_file[(new_file['기간']<2017)].reset_index(drop=True)
df_2017 = new_file[(new_file['기간']>=2017)].reset_index(drop=True)

In [12]:
df_2016.to_csv('data/2016 이전/서울시 5대 범죄 발생현황 연도별 통계(~2016 cp949).csv', encoding='cp949', index=False)
df_2017.to_csv('data/서울시 5대 범죄 발생현황 연도별 통계(2017~2019, cp949).csv', encoding='cp949', index=False)

### 2. 관서별 5대범죄 발생 및 검거 현황(2000~2019)
#### 1) 2000~2019년 데이터 합치기

In [16]:
input_file = r'C:\Users\HYEONJI\acorn\FinalProject\raw\관서별 5대범죄 발생 및 검거 현황(2000~2019)'
output_file = r'C:\Users\HYEONJI\acorn\FinalProject\raw\관서별 5대범죄 발생 및 검거 현황(2000~2019).csv'

allFile_list = glob.glob(os.path.join(input_file, '*'))
allData = []
year = 2000

# 컬럼명 동일화, '건수' 컬럼 타입 동일화, 년도 컬럼 추가
for file in allFile_list:
    df = pd.read_csv(file, encoding='cp949')
    df = df.iloc[:310, :4]
    df.columns=['관서', '죄종', '발생검거', '건수']
    df.astype({'건수' : 'int'})
    df['년'] = year
    allData.append(df)
    year+=1

dataCombine = pd.concat(allData, axis=0)
dataCombine.to_csv(output_file, index=False, encoding='cp949')

[]

In [17]:
# 저장된 데이터 확인
df = pd.read_csv('raw/관서별 5대범죄 발생 및 검거 현황(2000~2019).csv', encoding='cp949')
df

,관서,죄종,발생검거,건수,년
0,중부,살인,발생,1.0,2000
1,중부,살인,검거,1.0,2000
2,중부,강도,발생,17.0,2000
3,중부,강도,검거,15.0,2000
4,중부,강간,발생,14.0,2000
...,...,...,...,...,...
6195,수서,강간,검거,151.0,2019
6196,수서,절도,발생,967.0,2019
6197,수서,절도,검거,461.0,2019
6198,수서,폭력,발생,1470.0,2019


#### 2) 관서별 주소, 구 추가

In [18]:
len(df['관서'].unique())

31

In [19]:
# 데이터 불러오기
addr = pd.read_csv('경찰서 주소.txt', delimiter = '\t', header=None)

# 컬럼명 변경
addr.rename(columns = {0:'경찰서', 1:'주소', 2:'홈페이지'}, inplace=True)

# 홈페이지 컬럼, '서울지방경찰서' row 삭제
addr.drop('홈페이지', axis=1, inplace=True)
addr.drop([0], axis=0, inplace=True)

# 구, 관서 컬럼 생성, 경찰서 컬럼 삭제
addr['구'] = addr.주소.str.split(' ').str[1]
addr['관서'] = addr['경찰서'].str[2:]
addr['관서'] = addr['관서'] .str[:-3]
addr.drop('경찰서', axis=1, inplace=True)
addr

,주소,구,관서
1,서울시 종로구 사직로8길 31,종로구,지방
2,서울시 강남구 테헤란로 114길 11,강남구,강남
3,서울시 강동구 성내로 57,강동구,강동
4,서울시 강북구 오패산로 406,강북구,강북
5,서울시 양천구 화곡로 73,양천구,강서
6,서울시 관악구 관악로5길 33,관악구,관악
7,서울시 광진구 자양로 167,광진구,광진
8,서울시 구로구 가마산로 235,구로구,구로
9,서울시 금천구 시흥대로73길 50,금천구,금천
10,서울시 중구 한강대로 410,중구,남대문


In [20]:
# 데이터 merge, 컬럼 순서 변경

po_addr = pd.merge(df, addr, how='left', on='관서')[['년', '관서', '주소', '구', '죄종', '발생검거', '건수']]
po_addr

,년,관서,주소,구,죄종,발생검거,건수
0,2000,중부,서울시 중구 수표로 27,중구,살인,발생,1.0
1,2000,중부,서울시 중구 수표로 27,중구,살인,검거,1.0
2,2000,중부,서울시 중구 수표로 27,중구,강도,발생,17.0
3,2000,중부,서울시 중구 수표로 27,중구,강도,검거,15.0
4,2000,중부,서울시 중구 수표로 27,중구,강간,발생,14.0
...,...,...,...,...,...,...,...
6195,2019,수서,서울시 강남구 개포로 617,강남구,강간,검거,151.0
6196,2019,수서,서울시 강남구 개포로 617,강남구,절도,발생,967.0
6197,2019,수서,서울시 강남구 개포로 617,강남구,절도,검거,461.0
6198,2019,수서,서울시 강남구 개포로 617,강남구,폭력,발생,1470.0


#### 3) 데이터 분리, 저장

In [22]:
# 2000~2016년
df_2016 = po_addr[po_addr['년']<=2016].reset_index(drop=True)
df_2016.to_csv('data/2016 이전/관서별 5대범죄 발생 및 검거 현황(2000~2016, cp949).csv', encoding='cp949', index=False)

In [23]:
# 2017~2019년
df_2019 = po_addr[po_addr['년']>2016].reset_index(drop=True)
df_2019.to_csv('data/관서별 5대범죄 발생 및 검거 현황(2017~2019, cp949).csv', encoding='cp949', index=False)

### 3. 관서별 살인, 폭력 죄종 범죄자 연령대 현황(2011~2019)
#### 1) 2011~2019년 데이터 합치기

In [13]:
input_file = r'C:\Users\HYEONJI\acorn\FinalProject\raw\관서별 살인, 폭력 죄종 범죄자 연령대 현황(2011~2019)'
output_file = r'C:\Users\HYEONJI\acorn\FinalProject\raw\관서별 살인, 폭력 죄종 범죄자 연령대 현황(2011~2019).csv'

allFile_list = glob.glob(os.path.join(input_file, '*'))
allData = []
year = 2011

# 년, 관서, 죄종 컬럼 생성
for file in allFile_list:
    df = pd.read_csv(file, encoding='cp949')
    df['년'] = year
    df['관서'] = df.구분.str.split(' ').str[0]
    df['죄종'] = df.구분.str.split(' ').str[1]
    allData.append(df)
    year+=1

dataCombine = pd.concat(allData, axis=0).drop('구분', axis=1)[['년', '관서', '죄종', '연령', '범죄자수']]
dataCombine.to_csv(output_file, index=False, encoding='cp949')

In [24]:
# 저장된 데이터 확인
df = pd.read_csv('raw/관서별 살인, 폭력 죄종 범죄자 연령대 현황(2011~2019).csv', encoding='cp949')
df

,년,관서,죄종,연령,범죄자수
0,2011,중부,살인,14미만,0
1,2011,중부,살인,14세,0
2,2011,중부,살인,15세,0
3,2011,중부,살인,16세,0
4,2011,중부,살인,17세,0
...,...,...,...,...,...
11155,2019,수서,폭력,56~60,155
11156,2019,수서,폭력,61~64,99
11157,2019,수서,폭력,65~70,82
11158,2019,수서,폭력,71이상,43


#### 2) 관서별 구 추가

In [25]:
# 데이터 merge, 컬럼 순서 변경

df_addr = pd.merge(df, addr, how='left', on='관서')[['년', '관서', '주소', '구', '죄종', '연령', '범죄자수']]
df_addr

,년,관서,주소,구,죄종,연령,범죄자수
0,2011,중부,서울시 중구 수표로 27,중구,살인,14미만,0
1,2011,중부,서울시 중구 수표로 27,중구,살인,14세,0
2,2011,중부,서울시 중구 수표로 27,중구,살인,15세,0
3,2011,중부,서울시 중구 수표로 27,중구,살인,16세,0
4,2011,중부,서울시 중구 수표로 27,중구,살인,17세,0
...,...,...,...,...,...,...,...
11155,2019,수서,서울시 강남구 개포로 617,강남구,폭력,56~60,155
11156,2019,수서,서울시 강남구 개포로 617,강남구,폭력,61~64,99
11157,2019,수서,서울시 강남구 개포로 617,강남구,폭력,65~70,82
11158,2019,수서,서울시 강남구 개포로 617,강남구,폭력,71이상,43


#### 3) 데이터 분리, 저장

In [26]:
# 2011~2016년
df_2016 = df_addr[df_addr['년']<=2016].reset_index(drop=True)
df_2016.to_csv('data/2016 이전/관서별 살인, 폭력 죄종 범죄자 연령대 현황(2011~2016, cp949).csv', encoding='cp949', index=False)

In [27]:
# 2017~2019년
df_2019 = df_addr[df_addr['년']>2016].reset_index(drop=True)
df_2019.to_csv('data/관서별 살인, 폭력 죄종 범죄자 연령대 현황(2017~2019, cp949).csv', encoding='cp949', index=False)

### 4. 서울시 청소년 5대범죄
#### 1) 2010~2019년 데이터 합치기

In [18]:
input_file = r'C:\Users\HYEONJI\acorn\FinalProject\raw\서울시 청소년 5대범죄'
output_file = r'C:\Users\HYEONJI\acorn\FinalProject\raw\서울시 청소년 5대범죄(2010~2019).csv'

allFile_list = glob.glob(os.path.join(input_file, '*'))
allData = []
year = 2010

# 년 컬럼 생성, 관서로 컬럼명 변경
for file in allFile_list:
    df = pd.read_csv(file, encoding='cp949')
    df.rename(columns={'구분' : '관서'}, inplace = True)
    df['년'] = year
    allData.append(df)
    year+=1

dataCombine = pd.concat(allData, axis=0)[['년', '관서', '살인', '강도', '강간·추행', '절도', '폭력']].reset_index(drop=True)

In [19]:
# Nan 값 삭제
dataCombine = dataCombine.iloc[:310, :]
dataCombine.isnull().sum()

년        0
관서       0
살인       0
강도       0
강간·추행    0
절도       0
폭력       0
dtype: int64

In [20]:
# 데이터 저장
dataCombine.to_csv(output_file, index=False, encoding='cp949')

In [35]:
# 저장된 데이터 확인
df = pd.read_csv('raw/서울시 청소년 5대범죄(2010~2019).csv', encoding='cp949')
df

,년,관서,살인,강도,강간·추행,절도,폭력
0,2010,중부,0,4,6.0,125.0,107.0
1,2010,종로,0,1,1.0,53.0,28.0
2,2010,남대문,0,0,1.0,19.0,23.0
3,2010,서대문,0,5,12.0,154.0,139.0
4,2010,혜화,0,6,5.0,124.0,40.0
...,...,...,...,...,...,...,...
305,2019,노원,0.0,1.0,22.0,112.0,195.0
306,2019,방배,0.0,0.0,0.0,19.0,22.0
307,2019,은평,0.0,0.0,16.0,134.0,110.0
308,2019,도봉,0.0,6.0,11.0,59.0,89.0


#### 2) 관서별 구 추가

In [36]:
# 데이터 merge, 컬럼 순서 변경

df_addr = pd.merge(df, addr, how='left', on='관서')[['년', '관서', '주소', '구', '살인', '강도', '강간·추행', '절도', '폭력']]
df_addr

,년,관서,주소,구,살인,강도,강간·추행,절도,폭력
0,2010,중부,서울시 중구 수표로 27,중구,0,4,6.0,125.0,107.0
1,2010,종로,서울시 종로구 율곡로 46,종로구,0,1,1.0,53.0,28.0
2,2010,남대문,서울시 중구 한강대로 410,중구,0,0,1.0,19.0,23.0
3,2010,서대문,서울 서대문구 통일로 113,서대문구,0,5,12.0,154.0,139.0
4,2010,혜화,서울시 종로구 창경궁로 112-16,종로구,0,6,5.0,124.0,40.0
...,...,...,...,...,...,...,...,...,...
305,2019,노원,서울시 노원구 노원로 283,노원구,0.0,1.0,22.0,112.0,195.0
306,2019,방배,서울시 서초구 동작대로 204,서초구,0.0,0.0,0.0,19.0,22.0
307,2019,은평,서울시 은평구 연서로 365,은평구,0.0,0.0,16.0,134.0,110.0
308,2019,도봉,서울시 도봉구 노해로 403,도봉구,0.0,6.0,11.0,59.0,89.0


#### 3) 데이터 분리 및 저장

In [30]:
# 2010~2014년
df_2016 = df_addr[df_addr['년']<=2016].reset_index(drop=True)
df_2016.to_csv('data/2016 이전/서울시 청소년 5대범죄(2010~2016, cp949).csv', encoding='cp949', index=False)

In [31]:
# 2015~2019년
df_2019 = df_addr[df_addr['년']>2016].reset_index(drop=True)
df_2019.to_csv('data/서울시 청소년 5대범죄(2017~2019, cp949).csv', encoding='cp949', index=False)